<a href="https://colab.research.google.com/github/EPRADDH/NLP_Natural_Language_Processing_Methods/blob/main/Extracting_Keywords_with_TF_IDF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# You may be wondering why you would even use TF-IDF for any task at all ?

The truth is TF-IDF is easy to understand, easy to compute and is one of the most versatile statistic that shows the relative importance of a word or phrase in a document or a set of documents in comparison to the rest of your corpus.

TF-IDF can be used for a wide range of tasks including text classification, clustering / topic-modeling, search, keyword extraction and a whole lot more.

In this article, we will see how to use TF-IDF from the scikit-learn package to extract keywords from documents.

[TF-IDF  videos online](https://www.youtube.com/results?search_query=tf-idf.)

we’ll use a stack overflow dataset for Keyword extraction.
We will be using two files

1.stackoverflow-data-idf.json has 20,000 posts

2.stackoverflow-test.json has 500 posts 

where first one used to compute the Inverse Document Frequency (IDF) and  another one, we would use  as a test set for us to extract keywords from.  

In [ ]:
#Drive Mount
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import json

In [ ]:
import pandas as pd

# read json into a dataframe Here, lines=True simply means we are treating each line in the text file as a separate json string.

df_idf=pd.read_json("/content/drive/MyDrive/NLP-Natural-Language-Processing-Methods/keyword extraction using TFIDF/stackoverflow-data-idf.json",lines=True,encoding='cp1252')

In [ ]:
df_idf.head(2)

,id,title,body,answer_count,comment_count,creation_date,last_activity_date,last_editor_display_name,owner_display_name,owner_user_id,post_type_id,score,tags,view_count,accepted_answer_id,favorite_count,last_edit_date,last_editor_user_id,community_owned_date
0,4821394,Serializing a private struct - Can it be done?,<p>I have a public class that contains a priva...,1,0,2011-01-27 20:19:13.563 UTC,2011-01-27 20:21:37.59 UTC,,,163534.0,1,0,c#|serialization|xml-serialization,296,NaN,NaN,NaN,NaN,NaN
1,3367882,How do I prevent floated-right content from ov...,<p>I have the following HTML:</p>\n\n<pre><cod...,2,2,2010-07-30 00:01:50.9 UTC,2012-05-10 14:16:05.143 UTC,,,1190.0,1,2,css|overflow|css-float|crop,4121,3367943.0,0.0,2012-05-10 14:16:05.143 UTC,44390.0,NaN


In [ ]:
# print schema
print("Schema:\n\n",df_idf.dtypes)
print("Number of questions,columns=",df_idf.shape)

Schema:

 id                            int64
title                        object
body                         object
answer_count                  int64
comment_count                 int64
creation_date                object
last_activity_date           object
last_editor_display_name     object
owner_display_name           object
owner_user_id               float64
post_type_id                  int64
score                         int64
tags                         object
view_count                    int64
accepted_answer_id          float64
favorite_count              float64
last_edit_date               object
last_editor_user_id         float64
community_owned_date         object
dtype: object
Number of questions,columns= (20000, 19)


this dataset contains 19 fields.but we only intrusted in the body and title which will become our source of text for keyword extraction.

We will now create a field that combines both body and title so we have it in one field. We will also print the second text entry in our new field just to see what the text looks like.

In [ ]:
import re
def pre_process(text):
    
    # lowercase
    text=text.lower()

    #remove tags
    text=re.sub("&lt;/?.*?&gt;"," &lt;&gt; ",text)
    
    
    #remove tags
    text=re.sub("","",text)
    
    # remove special characters and digits
    text=re.sub("(\\d|\\W)+"," ",text)
    
    return text

In [ ]:
df_idf['text'] = df_idf['title'] + df_idf['body']
df_idf['text'] = df_idf['text'].apply(lambda x:pre_process(x))

In [ ]:
#show the 'text' to check how it is
df_idf['text'][3]

'loop variable as parameter in asynchronous function call p i have an object with the following form p pre code sortedfilters task appointment email code pre p now i want to use a code for in code loop to build my tasks array for code async parallel code which contains the functions to be executed asynchronously p pre code var tasks for var entity in sortedfilters tasks push function callback var entityresult fetchrecordsforentity entity businessunits sortedfilters var formattedentityresults formatresults entity entityresult callback null formattedentityresults code pre p the problem is that at the moment the functions are called code entity code points at the last value of the loop in this case it would be code email code p p how can i nail the exact value at the moment the function is added to the array p '

All of the cleaning happens in pre_process(..). You can do a lot more stuff in pre_process(..), such as eliminate all code sections, normalize the words to its root, etc, but for simplicity we perform only some mild pre-processing.

# Creating Vocabulary and Word Counts for IDF

Now we need to create the vocabulary and start the counting process. We can use the CountVectorizer to create a vocabulary from all the text in our df_idf['text']

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
import re

def get_stop_words(stop_file_path):
    """load stop words """
    
    with open(stop_file_path, 'r', encoding="utf-8") as f:
        stopwords = f.readlines()
        stop_set = set(m.strip() for m in stopwords)
        return frozenset(stop_set)

In [ ]:
#load a set of stop words
#stopwords=get_stop_words("resources/stopwords.txt")
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
from nltk.corpus import stopwords
sw = stopwords.words("english")

In [ ]:
#get the text column 
docs=df_idf['text'].tolist()

docs[3]

'loop variable as parameter in asynchronous function call p i have an object with the following form p pre code sortedfilters task appointment email code pre p now i want to use a code for in code loop to build my tasks array for code async parallel code which contains the functions to be executed asynchronously p pre code var tasks for var entity in sortedfilters tasks push function callback var entityresult fetchrecordsforentity entity businessunits sortedfilters var formattedentityresults formatresults entity entityresult callback null formattedentityresults code pre p the problem is that at the moment the functions are called code entity code points at the last value of the loop in this case it would be code email code p p how can i nail the exact value at the moment the function is added to the array p '

# create a vocabulary

In [ ]:
#create a vocabulary of words, 
#ignore words that appear in 85% of documents, 
#eliminate stop words
cv=CountVectorizer(max_df=0.85,stop_words=sw)
word_count_vector=cv.fit_transform(docs)

While cv.fit(...) would only create the vocabulary, cv.fit_transform(...) creates the vocabulary and returns a term-document matrix which is what we want.in this, each column in the matrix represents a word in the vocabulary while each row represents the document in our dataset where the values in this case are the word counts. Note that with this representation, counts of some words could be 0 if the word did not appear in the corresponding document

In [ ]:
word_count_vector.shape

(20000, 121098)

The resulting shape of word_count_vector is (20000,127626) since we have 20,000 documents in our dataset (the rows) and the vocabulary size is 127626. In some text mining applications such as clustering and text classification we typically limit the size of the vocabulary. It’s really easy to do this by setting max_features=vocab_size when instantiating CountVectorizer. For this tutorial let’s limit our vocabulary size to 10,000.

In [ ]:
cv=CountVectorizer(max_df=0.85,stop_words=sw,max_features=10000)
word_count_vector=cv.fit_transform(docs)


In [ ]:
word_count_vector.shape

(20000, 10000)

Now, let’s look at 50 words from our vocabulary.

In [ ]:
list(cv.vocabulary_.keys())[:50]

['serializing',
 'private',
 'struct',
 'done',
 'public',
 'class',
 'contains',
 'properties',
 'mostly',
 'string',
 'want',
 'serialize',
 'attempt',
 'stream',
 'disk',
 'using',
 'xmlserializer',
 'get',
 'error',
 'saying',
 'types',
 'serialized',
 'need',
 'way',
 'keep',
 'prevent',
 'floated',
 'right',
 'content',
 'overlapping',
 'main',
 'following',
 'html',
 'pre',
 'code',
 'lt',
 'gt',
 'long',
 'fit',
 'space',
 'cut',
 'display',
 'follows',
 'icon',
 'css',
 'td',
 'span',
 'overflow',
 'hidden',
 'white']

# TfidfTransformer to Compute Inverse Document Frequency (IDF)
It’s now time to compute the IDF values. In the code below, we are essentially taking the sparse matrix from CountVectorizer (word_count_vector) to generate the IDF when you invoke tfidf_transformer.fit(...)

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer

tfidf_transformer=TfidfTransformer(smooth_idf=True,use_idf=True)
tfidf_transformer.fit(word_count_vector)

TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)

In [ ]:
# print idf values 
df_idf = pd.DataFrame(tfidf_transformer.idf_, index=cv.get_feature_names(),columns=["idf_weights"]) 
 
# sort ascending 
df_idf.sort_values(by=['idf_weights'])

,idf_weights
code,1.242695
pre,1.373653
using,2.015471
gt,2.028950
lt,2.117845
...,...
itargetspeed,10.210390
row_name,10.210390
item_level,10.210390
itick,10.210390


# Computing TF-IDF and Extracting Keywords
The important point to note here is that the IDF should always be based on a large corpora and should be representative of texts you would be using to extract keywords.

Once we have our IDF computed, we are now ready to compute TF-IDF and then extract top keywords from the TF-IDF vectors.

here we will extract top keywords for the questions in data/stackoverflow-test.json. This data file has 500 questions with fields identical to that of data/stackoverflow-data-idf.json as we saw above. We will start by reading our test file, extracting the necessary fields (title and body) and getting the texts into a list.

In [ ]:
# read test docs into a dataframe and concatenate title and body
df_test=pd.read_json("/content/drive/MyDrive/NLP-Natural-Language-Processing-Methods/keyword extraction using TFIDF/stackoverflow-test.json",lines=True,encoding='cp1252')
df_test['text'] = df_test['title'] + df_test['body']
df_test['text'] =df_test['text'].apply(lambda x:pre_process(x))

# get test docs into a list
docs_test=df_test['text'].tolist()
docs_title=df_test['title'].tolist()
docs_body=df_test['body'].tolist()

In [ ]:
docs_test[:4]

['integrate war plugin for m eclipse into eclipse project p i set up a small web project with jsf and maven now i want to deploy on a tomcat server is there a possibility to automate that like a button in eclipse that automatically deploys the project to tomcat p p i read about a the a href http maven apache org plugins maven war plugin rel nofollow noreferrer maven war plugin a but i couldn t find a tutorial how to integrate that into my process eclipse m eclipse p p can you link me to help or try to explain it thanks p ',
 'phantomjs node page evaulate seems to hang p i have an implementation of waitfor with phantomjs node and it seems that the code sitepage evaluate code has a big lag compared to when it should evaluate true you ll see below that i m logging out the content value and the content logs with what should evaluate as true but this doesn t seem to occur for a good seconds or so after the fact p p any idea what would cause this delay or if there s a better way to evaluate 

The next step is to compute the tf-idf value for a given document in our test set by invoking tfidf_transformer.transform(...). This generates a vector of tf-idf scores. Next, we sort the words in the vector in descending order of tf-idf values and then iterate over to extract the top-n keywords. In the example below, we are extracting keywords for the first document in our test set.

In [ ]:
def sort_coo(coo_matrix):
    tuples = zip(coo_matrix.col, coo_matrix.data)
    return sorted(tuples, key=lambda x: (x[1], x[0]), reverse=True)

def extract_topn_from_vector(feature_names, sorted_items, topn=10):
    """get the feature names and tf-idf score of top n items"""
    
    #use only topn items from vector
    sorted_items = sorted_items[:topn]

    score_vals = []
    feature_vals = []
    
    # word index and corresponding tf-idf score
    for idx, score in sorted_items:
        
        #keep track of feature name and its corresponding score
        score_vals.append(round(score, 3))
        feature_vals.append(feature_names[idx])

    #create a tuples of feature,score
    #results = zip(feature_vals,score_vals)
    results= {}
    for idx in range(len(feature_vals)):
        results[feature_vals[idx]]=score_vals[idx]
    
    return results

The sort_coo(...) method essentially sorts the values in the vector while preserving the column index. Once you have the column index then its really easy to look-up the corresponding word value as you would see in extract_topn_from_vector(...)

In [ ]:
# we only needs to do this once, this is a mapping of index to 
feature_names=cv.get_feature_names()

# get the document that we want to extract keywords from
doc=docs_test[0]

In [ ]:
#generate tf-idf for the given document
tf_idf_vector=tfidf_transformer.transform(cv.transform([doc]))

In [ ]:
#sort the tf-idf vectors by descending order of scores

sorted_items=sort_coo(tf_idf_vector.tocoo())

In [ ]:
#extract only the top n; n here is 10
keywords=extract_topn_from_vector(feature_names,sorted_items,10)


In [ ]:
# now print the results
print("\n=====Doc=====")
print(doc)
print("\n===Keywords===")
for k in keywords:
    print(k,keywords[k])


# now print the results
print("\n=====Title=====")
print(docs_title[0])
print("\n=====Body=====")
print(docs_body[0])
print("\n===Keywords===")
for k in keywords:
    print(k,keywords[k])


=====Doc=====
integrate war plugin for m eclipse into eclipse project p i set up a small web project with jsf and maven now i want to deploy on a tomcat server is there a possibility to automate that like a button in eclipse that automatically deploys the project to tomcat p p i read about a the a href http maven apache org plugins maven war plugin rel nofollow noreferrer maven war plugin a but i couldn t find a tutorial how to integrate that into my process eclipse m eclipse p p can you link me to help or try to explain it thanks p 

===Keywords===
eclipse 0.492
maven 0.453
war 0.395
plugin 0.267
integrate 0.234
tomcat 0.224
project 0.198
automate 0.131
jsf 0.126
possibility 0.122

=====Title=====
Integrate War-Plugin for m2eclipse into Eclipse Project

=====Body=====
<p>I set up a small web project with JSF and Maven. Now I want to deploy on a Tomcat server. Is there a possibility to automate that like a button in Eclipse that automatically deploys the project to Tomcat?</p>

<p>I r

In [ ]:

# put the common code into several methods
def get_keywords(idx):

    #generate tf-idf for the given document
    tf_idf_vector=tfidf_transformer.transform(cv.transform([docs_test[idx]]))

    #sort the tf-idf vectors by descending order of scores
    sorted_items=sort_coo(tf_idf_vector.tocoo())

    #extract only the top n; n here is 10
    keywords=extract_topn_from_vector(feature_names,sorted_items,20)
    
    return keywords

def print_results(idx,keywords):
    # now print the results
    print("\n=====Title=====")
    print(docs_title[idx])
    print("\n=====Body=====")
    print(docs_body[idx])
    print("\n===Keywords===")
    for k in keywords:
        print(k,keywords[k])

In [ ]:
#Now let's look at keywords generated for a much longer question:

idx=120
keywords=get_keywords(idx)
print_results(idx,keywords)


=====Title=====
SQL Import Wizard - Error

=====Body=====
<p>I have a CSV file that I'm trying to import into SQL Management Server Studio.</p>

<p>In Excel, the column giving me trouble looks like this:
<a href="https://i.stack.imgur.com/pm0uS.png" rel="nofollow noreferrer"><img src="https://i.stack.imgur.com/pm0uS.png" alt="enter image description here"></a></p>

<p>Tasks > import data > Flat Source File > select file</p>

<p><a href="https://i.stack.imgur.com/G4b6I.png" rel="nofollow noreferrer"><img src="https://i.stack.imgur.com/G4b6I.png" alt="enter image description here"></a></p>

<p>I set the data type for this column to DT_NUMERIC, adjust the DataScale to 2 in order to get 2 decimal places, but when I click over to Preview, I see that it's clearly not recognizing the numbers appropriately:</p>

<p><a href="https://i.stack.imgur.com/NZhiQ.png" rel="nofollow noreferrer"><img src="https://i.stack.imgur.com/NZhiQ.png" alt="enter image description here"></a></p>

<p>The column ma

In [ ]:
idx=122
keywords=get_keywords(idx)
print_results(idx,keywords)


=====Title=====
Retrieving server address from email

=====Body=====
<p>How can I retrieve mail's SMTP server from email (for example oneat@op.pl) ?</p>

===Keywords===
email 0.452
smtp 0.361
op 0.352
pl 0.34
retrieving 0.326
server 0.322
mail 0.286
retrieve 0.251
address 0.225
example 0.146


#Generate keywords for a batch of documents

In [ ]:
#generate tf-idf for all documents in your list. docs_test has 500 documents
tf_idf_vector=tfidf_transformer.transform(cv.transform(docs_test))

results=[]
for i in range(tf_idf_vector.shape[0]):
    
    # get vector for a single document
    curr_vector=tf_idf_vector[i]
    
    #sort the tf-idf vector by descending order of scores
    sorted_items=sort_coo(curr_vector.tocoo())

    #extract only the top n; n here is 10
    keywords=extract_topn_from_vector(feature_names,sorted_items,10)
    
    
    results.append(keywords)

df=pd.DataFrame(zip(docs,results),columns=['doc','keywords'])
df

,doc,keywords
0,serializing a private struct can it be done p ...,"{'eclipse': 0.492, 'maven': 0.453, 'war': 0.39..."
1,how do i prevent floated right content from ov...,"{'evaluate': 0.464, 'content': 0.408, 'console..."
2,gradle command line p i m trying to run a shel...,"{'appdomain': 0.397, 'dynamic': 0.37, 'vs': 0...."
3,loop variable as parameter in asynchronous fun...,"{'background': 0.43, 'image': 0.397, 'jpg': 0...."
4,canot get the href value p hi i need to valid ...,"{'uri': 0.369, 'bitmap': 0.315, 'intent': 0.30..."
...,...,...
495,how to unbind click and click submit button in...,"{'delphi': 0.685, 'compatible': 0.31, 'win': 0..."
496,swaggerui auth redirect swaggeruiauth of null ...,"{'node': 0.515, 'null': 0.328, 'selectsingleno..."
497,ssrs value display error for ssrs conditional ...,"{'logo': 0.539, 'triangle': 0.313, 'step': 0.3..."
498,accessing and changing a class instance from a...,"{'code': 0.452, 'length': 0.344, 'ev': 0.334, ..."


#What is Keyword Extraction?
Keyword extraction is defined as the task of Natural language processing that automatically identifies a set of terms to describe the subject of the text. This is an important method in information retrieval (IR) systems: keywords simplify and speed up research. Keyword extraction can be used to reduce text dimensionality for further text analysis (subject modeling text classification)

lets take one more task on Keyword Extraction

In [ ]:
import pandas as pd # data processing
df = pd.read_csv('C:/Users/Pradeep Dhote/Documents/NLP/NLP_Natural_Language_Processing_Methods/keyword_extraction_using_tfidf/papers.csv')

FileNotFoundError: ignored

In [ ]:
import pandas as pd

Dataset To upload from your local drive,

In [ ]:
from google.colab import files
uploaded = files.upload()

import io
df2 = pd.read_csv(io.BytesIO(uploaded['papers.csv']))
# Dataset is now stored in a Pandas Dataframe

Saving papers.csv to papers (1).csv


KeyboardInterrupt: ignored

In [ ]:
import pandas as pd # data processing
df2 = pd.read_csv('/content/papers.csv')

In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [ ]:
 import re
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer

stop_words = set(stopwords.words('english'))
##Creating a list of custom stopwords
new_words = ["fig","figure","image","sample","using", 
             "show", "result", "large", 
             "also", "one", "two", "three", 
             "four", "five", "seven","eight","nine"]
stop_words = list(stop_words.union(new_words))

In [ ]:
def pre_process(text):
    
    # lowercase
    text=text.lower()
    
    #remove tags
    text=re.sub("&lt;/?.*?&gt;"," &lt;&gt; ",text)
    
    # remove special characters and digits
    text=re.sub("(\\d|\\W)+"," ",text)
    
    ##Convert to list from string
    text = text.split()
    
    # remove stopwords
    text = [word for word in text if word not in stop_words]

    # remove words less than three letters
    text = [word for word in text if len(word) >= 3]

    # lemmatize
    lmtzr = WordNetLemmatizer()
    text = [lmtzr.lemmatize(word) for word in text]
    
    return ' '.join(text)
docs = df2['paper_text'].apply(lambda x:pre_process(x))

#Using TF-IDF

Using the tf-idf weighting scheme, the keywords are the words with the highest TF-IDF score. For this task, I’ll first use the CountVectorizer method in Scikit-learn to create a vocabulary and generate the word count:

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
#docs = docs.tolist()
#create a vocabulary of words, 
cv=CountVectorizer(max_df=0.95,         # ignore words that appear in 95% of documents
                   max_features=10000,  # the size of the vocabulary
                   ngram_range=(1,3)    # vocabulary contains single words, bigrams, trigrams
                  )
word_count_vector=cv.fit_transform(docs)

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer

tfidf_transformer=TfidfTransformer(smooth_idf=True,use_idf=True)
tfidf_transformer.fit(word_count_vector)

TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)

Now, we are ready for the final step. In this step, I will create a function for the task of Keyword Extraction with Python by using the Tf-IDF vectorization:

In [ ]:
def sort_coo(coo_matrix):
    tuples = zip(coo_matrix.col, coo_matrix.data)
    return sorted(tuples, key=lambda x: (x[1], x[0]), reverse=True)

def extract_topn_from_vector(feature_names, sorted_items, topn=10):
    """get the feature names and tf-idf score of top n items"""
    
    #use only topn items from vector
    sorted_items = sorted_items[:topn]

    score_vals = []
    feature_vals = []

    for idx, score in sorted_items:
        fname = feature_names[idx]
        
        #keep track of feature name and its corresponding score
        score_vals.append(round(score, 3))
        feature_vals.append(feature_names[idx])

    #create a tuples of feature,score
    #results = zip(feature_vals,score_vals)
    results= {}
    for idx in range(len(feature_vals)):
        results[feature_vals[idx]]=score_vals[idx]
    
    return results

In [ ]:
# get feature names
feature_names=cv.get_feature_names()

In [ ]:
def get_keywords(idx, docs):

    #generate tf-idf for the given document
    tf_idf_vector=tfidf_transformer.transform(cv.transform([docs[idx]]))

    #sort the tf-idf vectors by descending order of scores
    sorted_items=sort_coo(tf_idf_vector.tocoo())

    #extract only the top n; n here is 10
    keywords=extract_topn_from_vector(feature_names,sorted_items,10)
    
    return keywords

In [ ]:
def print_results(idx,keywords, df2):
    # now print the results
    print("\n=====Title=====")
    print(df2['title'][idx])
    print("\n=====Abstract=====")
    print(df2['abstract'][idx])
    print("\n===Keywords===")
    for k in keywords:
        print(k,keywords[k])

In [ ]:
idx=999
keywords=get_keywords(idx, docs)
print_results(idx,keywords, df2)


=====Title=====
Shape Context: A New Descriptor for Shape Matching and Object Recognition

=====Abstract=====
Abstract Missing

===Keywords===
shape 0.686
matching 0.249
prototype 0.24
descriptor 0.201
point 0.188
object 0.167
context 0.157
distance 0.119
correspondence 0.111
recognition 0.104
